In [14]:
from tg.grammar_ru.ml.corpus import CorpusReader
from tg.grammar_ru.common import Loc
from pathlib import Path
reader = CorpusReader(Path(Loc.corpus_path/'example.base.zip')).get_frames()

In [15]:
from tg.common._common import DataBundle
from tg.grammar_ru.algorithms import AntecedentCandidatesAlgorithm

first_df = reader.first()
alg = AntecedentCandidatesAlgorithm(max_candidates=5)
bundle = DataBundle(**{'src': first_df})
simple_index = alg.get_candidates(bundle)
simple_index

,pronoun_word_id,candidate_word_id,candidate_distance
0,10,0,0
1,18,0,0
2,30,0,0
3,48,3,3
4,48,6,2
...,...,...,...
208,921,896,4
209,921,907,3
210,921,914,2
211,921,918,1


In [16]:
right_pair_ids = [0, 1, 2, 6, 7, 8, 9, 11, 15, 19, 27, 32, 37, 42, 45, 47, 49, 51, 53, 57, 58, 62, 65, 72, 74, 77, 79, 87, 
                  91, 93, 96, 100, 101, 105, 106, 109, 110, 112, 113, 115, 117, 119, 121, 127, 129, 131, 136, 137, 139, 142, 
                  144, 147, 148, 151, 157, 159, 163, 167, 168, 172, 174, 176, 179, 181, 190, 194, 200, 202, 203, 205, 208]

In [22]:
bundle['pymorphy']

,normal_form,alternatives,score,delta_score,POS,animacy,gender,number,case,aspect,transitivity,person,tense,mood,voice,involvement
word_id,,,,,,,,,,,,,,,,
0,лилия,2,0.666666,0.333333,NOUN,anim,femn,sing,nomn,NaN,NaN,NaN,NaN,NaN,NaN,None
1,очнуться,1,1.000000,1.000000,VERB,NaN,femn,sing,NaN,perf,intr,NaN,past,indc,NaN,None
2,от,1,1.000000,1.000000,PREP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,суетливый,3,0.333333,0.000000,ADJF,NaN,masc,sing,gent,NaN,NaN,NaN,NaN,NaN,NaN,None
4,",",1,1.000000,1.000000,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921,она,1,1.000000,1.000000,NPRO,NaN,femn,sing,nomn,NaN,NaN,3per,NaN,NaN,NaN,None
922,кубарем,2,0.666666,0.333333,ADVB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
923,покатиться,1,1.000000,1.000000,VERB,NaN,femn,sing,NaN,perf,intr,NaN,past,indc,NaN,None


In [17]:
simple_index['is_match'] = simple_index.index.isin(right_pair_ids)
simple_index

,pronoun_word_id,candidate_word_id,candidate_distance,is_match
0,10,0,0,True
1,18,0,0,True
2,30,0,0,True
3,48,3,3,False
4,48,6,2,False
...,...,...,...,...
208,921,896,4,True
209,921,907,3,False
210,921,914,2,False
211,921,918,1,False


In [18]:
import pandas as pd

copied = simple_index.copy()
groups = copied.groupby('pronoun_word_id')
dfs = []
for pronoun_id,group in groups:
    crossed = group.merge(group, how='cross')
    crossed = crossed[crossed['is_match_x'] != crossed['is_match_y']].reset_index(drop=True)
    crossed = (crossed
               .rename(columns={"pronoun_word_id_x": "pronoun_word_id", "B": "c"})
               .drop(columns=['is_match_x', 'pronoun_word_id_y'])
               .drop(labels=range(len(crossed) // 2), axis=0))
    dfs.append(crossed)
compare_index = pd.concat(dfs, axis=0, ignore_index=True).reset_index(drop=True)
compare_index

,pronoun_word_id,candidate_word_id_x,candidate_distance_x,candidate_word_id_y,candidate_distance_y,is_match_y
0,48,38,0,3,3,False
1,48,38,0,6,2,False
2,48,38,0,34,1,False
3,83,65,1,61,2,False
4,83,65,1,82,0,False
...,...,...,...,...,...,...
202,917,914,0,896,2,True
203,921,907,3,896,4,True
204,921,914,2,896,4,True
205,921,918,1,896,4,True


In [23]:
bundle_simple = bundle.copy()
bundle_simple['index'] = simple_index
bundle_compare = bundle.copy()
bundle_simple['index'] = compare_index

In [24]:
bundle_simple

{'src': {'shape': (926, 10), 'index_name': None}, 'pymorphy': {'shape': (926, 16), 'index_name': 'word_id'}, 'slovnet': {'shape': (926, 17), 'index_name': 'word_id'}, 'index': {'shape': (207, 6), 'index_name': None}}